In [4]:
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [28]:
m = folium.Map(location=[-34.679725714625725, -58.45085487390574], zoom_start =14)

In [29]:
m.save("primermapa.html")

In [33]:
tooltip = "Haz click"
folium.Marker([-34.679725714625725, -58.45085487390574], popup="ETEC", tooltip=tooltip, icon= folium.Icon(color="red",icon="info-sign")).add_to(m)
m.save("primermapa.html")

In [31]:
folium.Circle(
    radius = 250,
    location = [-34.679725714625725, -58.45085487390574],
    popup = "Zona de medicion",
    color = "#3186cc",
    fill = "#3186cc"
).add_to(m)
m.save("primermapa.html")

In [5]:
df = pd.read_csv("../data/mediciones.csv")

In [6]:
df["fecha_hora"] = pd.to_datetime(df["fecha_hora"])

In [7]:
zonas = df["zona"].unique()
zona_widget = widgets.Dropdown(options=zonas, description="Zona:")
fecha_inicio_widget = widgets.DatePicker(
    description="Desde:", value=df["fecha_hora"].min().date()
)
fecha_fin_widget = widgets.DatePicker(
    description="Hasta:", value=df["fecha_hora"].max().date()
)
boton_actualizar = widgets.Button(description="Actualizar", button_style='info')

In [8]:
def actualizar(_):
    clear_output(wait=True)
    display(zona_widget, fecha_inicio_widget, fecha_fin_widget, boton_actualizar)

    zona_seleccionada = zona_widget.value
    fecha_inicio = fecha_inicio_widget.value
    fecha_fin = fecha_fin_widget.value

    df_filtrado = df[
        (df["zona"] == zona_seleccionada)
        & (df["fecha_hora"].dt.date >= fecha_inicio)
        & (df["fecha_hora"].dt.date <= fecha_fin)
    ]

    print(f"\n📍 Datos de {zona_seleccionada} entre {fecha_inicio} y {fecha_fin}\n")
    display(df_filtrado)

    if not df_filtrado.empty:
        print("\n📊 Estadísticas del período seleccionado:")
        print(f"Temperatura máxima: {df_filtrado['temperatura'].max():.2f} °C")
        print(f"Temperatura promedio: {df_filtrado['temperatura'].mean():.2f} °C")
        print(f"Temperatura mínima: {df_filtrado['temperatura'].min():.2f} °C")
        print(f"CO₂ promedio: {df_filtrado['co2'].mean():.2f} ppm")

        plt.figure(figsize=(8, 4))
        plt.plot(df_filtrado["fecha_hora"], df_filtrado["co2"], label="CO₂ (ppm)", marker="o")
        plt.plot(df_filtrado["fecha_hora"], df_filtrado["temperatura"], label="Temperatura (°C)", marker="s")
        plt.xlabel("Fecha")
        plt.ylabel("Valores")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("No hay datos en el rango seleccionado.")

In [9]:
boton_actualizar.on_click(actualizar)


In [15]:
display(zona_widget, fecha_inicio_widget, fecha_fin_widget, boton_actualizar)

Dropdown(description='Zona:', options=('ETEC',), value='ETEC')

DatePicker(value=datetime.date(2025, 11, 11), description='Desde:', step=1)

DatePicker(value=datetime.date(2025, 11, 11), description='Hasta:', step=1)

Button(button_style='info', description='Actualizar', style=ButtonStyle())